In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model= "llama3.2:1b")

In [2]:
from langchain_core.prompts import PromptTemplate

prompt_template =PromptTemplate(
    template = "What is the capital of {country}?",
    input_variables = ["country"],
)

prompt = prompt_template.invoke({"country":"France"})

print(prompt)

llm.invoke(prompt_template.invoke({"country":"France"}))


text='What is the capital of France?'


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:54:08.495668Z', 'done': True, 'done_reason': 'stop', 'total_duration': 181825375, 'load_duration': 71502542, 'prompt_eval_count': 32, 'prompt_eval_duration': 56012833, 'eval_count': 8, 'eval_duration': 53823375, 'model_name': 'llama3.2:1b'}, id='run--3b411099-5c0e-4001-b46e-e50b4d9ece78-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

message_list=[
    SystemMessage(content="You are a helpful assistant that can answer questions about the capital of countries."),
    HumanMessage(content="What is the capital of South Korea?"),
    AIMessage(content="The capital of South Korea is Seoul."),
    HumanMessage(content="What is the capital of South Korea?")
]
llm.invoke(message_list)


AIMessage(content="The answer is still the same: Seoul, which is the country's largest city and administrative center.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:54:08.718375Z', 'done': True, 'done_reason': 'stop', 'total_duration': 214109459, 'load_duration': 46649334, 'prompt_eval_count': 74, 'prompt_eval_duration': 37529834, 'eval_count': 21, 'eval_duration': 129143583, 'model_name': 'llama3.2:1b'}, id='run--70c7a24d-7949-435b-af84-9f357b85496f-0', usage_metadata={'input_tokens': 74, 'output_tokens': 21, 'total_tokens': 95})

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt_template =ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant that can answer questions about the capital of countries."
    ),
    (
        "human",
        "What is the capital of {country}?"
    ),

])

chat_prompt = chat_prompt_template.invoke({"country":"France"})
print(chat_prompt)
print("==================")
ai_message = llm.invoke(chat_prompt_template.invoke({"country":"France"}))
print(ai_message)
print("==================")
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"France"})))




messages=[SystemMessage(content='You are a helpful assistant that can answer questions about the capital of countries.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:54:08.84096Z', 'done': True, 'done_reason': 'stop', 'total_duration': 104587125, 'load_duration': 47498500, 'prompt_eval_count': 47, 'prompt_eval_duration': 12689459, 'eval_count': 8, 'eval_duration': 43855458, 'model_name': 'llama3.2:1b'} id='run--04460450-06cc-44f1-8e9d-01eac20d3ddd-0' usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55}


In [5]:
ai_message

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:54:08.84096Z', 'done': True, 'done_reason': 'stop', 'total_duration': 104587125, 'load_duration': 47498500, 'prompt_eval_count': 47, 'prompt_eval_duration': 12689459, 'eval_count': 8, 'eval_duration': 43855458, 'model_name': 'llama3.2:1b'}, id='run--04460450-06cc-44f1-8e9d-01eac20d3ddd-0', usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55})

In [6]:
answer

'The capital of France is Paris.'

In [7]:
#아래보다 다음과 같은방식이 더 정석이다 / JsonOutputParser는 사용하지않는다
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    area: float = Field(description="The area of the country")
    currency: str = Field(description="The currency of the country")
    language: str = Field(description="The language of the country")

structured_llm = llm.with_structured_output(CountryDetail)

In [8]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt=PromptTemplate(
    template="""Given following information about {country}:
    What is the capital of {country}?
    What is the population of {country}?
    What is the area of {country}?
    What is the currency of {country}?
    What is the language of {country}?
    What is the capital of {country}?
    What is the population of {country}?

    return it in JSON format and return the JSON dictionary only
    """,
    input_variables=["country"]
)

# country_detail_prompt.invoke({"country":"France"})

# output_parser = JsonOutputParser()
json_ai_message = structured_llm.invoke(country_detail_prompt.invoke({"country":"France"}))
# output_parser.invoke(json_ai_message)

In [9]:
json_ai_message

CountryDetail(capital='Paris', population=67, area=643.0, currency='Euro', language='French')

In [10]:
json_ai_message.capital

'Paris'

In [11]:
json_ai_message.population

67

In [12]:
json_ai_message.model_dump()

{'capital': 'Paris',
 'population': 67,
 'area': 643.0,
 'currency': 'Euro',
 'language': 'French'}

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt_template =ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant that can answer questions about the capital of countries."
    ),
    (
        "human",
        "What is the capital of {country}?"
    ),

])

chat_prompt = chat_prompt_template.invoke({"country":"France"})
print(chat_prompt)
print("==================")
ai_message = llm.invoke(chat_prompt_template.invoke({"country":"France"}))
print(ai_message)
print("==================")
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"France"})))




messages=[SystemMessage(content='You are a helpful assistant that can answer questions about the capital of countries.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]
content='The capital of France is Paris.' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-09-05T13:54:09.520286Z', 'done': True, 'done_reason': 'stop', 'total_duration': 109569792, 'load_duration': 46281958, 'prompt_eval_count': 47, 'prompt_eval_duration': 18711625, 'eval_count': 8, 'eval_duration': 44141917, 'model_name': 'llama3.2:1b'} id='run--f4eb1063-6352-48d6-92c1-40b88b27ddad-0' usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55}


In [ ]:
# MOving left to right =>
# Pipelining
chain = prompt_template | llm | output_parser


In [16]:
chain.invoke({"country":"France"})

'The capital of France is Paris.'

In [17]:
capital_chain = prompt_template | llm | output_parser
capital_chain.invoke({"country":"France"})

'The capital of France is Paris.'

In [ ]:
country_prompt = PromptTemplate(
    template="""
Guess the company based on the following information:
{information}
""",
input_variables=["information"]

)
country_chain = country_prompt | llm | output_parser
country_chain.invoke({"country":"France"})